#Titanic: Machine Learning from Disaster
Olivier RISSER-MAROIX (VieVie31)

In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
data_train = gl.load_sframe("train.csv")
data_test = gl.load_sframe("test.csv")

2016-03-19 12:56:17,024 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458388574.log


Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/train.csv

Parsing completed. Parsed 100 lines in 0.033549 secs.

This non-commercial license of GraphLab Create is assigned to orissermaroix@gmail.com and will expire on March 12, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/train.csv

...

Parsing completed. Parsed 100 lines in 0.007262 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/test.csv

Parsing completed. Parsed 418 lines in 0.009007 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Pclass	int
	Name	str
	Sex	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	Embarked	str

Rows: 3

Data:
+-------------+----------+--------+-------------------------------+--------+
| PassengerId | Survived | Pclass |              Name             |  Sex   |
+-------------+----------+--------+-------------------------------+--------+
|      1      |    0     |   3    |    Braund, Mr. Owen Harris    |  male  |
|      2      |    1     |   1    | Cumings, Mrs. John Bradley... | female |
|      3      |    1     |   3    |     Heikkinen, Miss. Laina    | female |
+-------------+----------+--------+-------------------------------+--------+
+------+-------+-------+------------------+---------+-------+----------+
| Age  | SibSp | Parch |      Ticket      |   Fare  | Cabin | Embarked |
+------+-------+-------+------------------+---------+-------+----------+
| 22.0 |   1   |   0   |    A/5 21171     |   7.25  |       |    S     |
| 38.0 |   1   |   0   |     PC 17599     | 71.2833 |  C85  |    C     |
| 26.0 |   0   |   0   | STON/O2. 3101282 |  7.925  |       |    S     |
+------+-------+-------+------------------+---------+-------+----------+
[3 rows x 12 columns]

##Cleanning trainning data

In [4]:
data_train["male"] = data_train["Sex"] == "male"
data_train["female"] = data_train["Sex"] == "female"
data_train = data_train.remove_column("Sex")

In [5]:
data_train["no_age"] = data_train["Age"] == None
data_train["Age"] = gl.SArray([0 if v == None else v for v in data_train["Age"]])

In [6]:
data_train["embarked_s"] = data_train["Embarked"] == "S"
data_train["embarked_c"] = data_train["Embarked"] == "C"
data_train["embarked_q"] = data_train["Embarked"] == "Q"
data_train["embarked_none"] = data_train["Embarked"] == None
data_train = data_train.remove_column("Embarked")

In [7]:
data_train["1_class"] = data_train["Pclass"] == 1
data_train["2_class"] = data_train["Pclass"] == 2
data_train["3_class"] = data_train["Pclass"] == 3
data_train = data_train.remove_column("Pclass")

In [8]:
gl.canvas.set_target("ipynb")
print data_train.head(3)
print data_train["Ticket"]
for v in data_train["Ticket"]:
    print v, " ",

+-------------+----------+-------------------------------+------+-------+-------+
| PassengerId | Survived |              Name             | Age  | SibSp | Parch |
+-------------+----------+-------------------------------+------+-------+-------+
|      1      |    0     |    Braund, Mr. Owen Harris    | 22.0 |   1   |   0   |
|      2      |    1     | Cumings, Mrs. John Bradley... | 38.0 |   1   |   0   |
|      3      |    1     |     Heikkinen, Miss. Laina    | 26.0 |   0   |   0   |
+-------------+----------+-------------------------------+------+-------+-------+
+------------------+---------+-------+------+--------+--------+------------+------------+
|      Ticket      |   Fare  | Cabin | male | female | no_age | embarked_s | embarked_c |
+------------------+---------+-------+------+--------+--------+------------+------------+
|    A/5 21171     |   7.25  |       |  1   |   0    |   0    |     1      |     0      |
|     PC 17599     | 71.2833 |  C85  |  0   |   1    |   0    |   

In [9]:
#processing the tickets numbers
#for try remove the non alpha numerics numbers
def toNumber(string):
    s = "0"
    for v in string:
        if v in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
            s += v
    return int(s)

data_train["PC"] = gl.SArray(["PC" in v for v in data_train["Ticket"]])
data_train["CA"] = gl.SArray(["CA" in v for v in data_train["Ticket"]])
data_train["C.A."] = gl.SArray(["C.A." in v for v in data_train["Ticket"]])
data_train["W./C."] = gl.SArray(["W./C." in v for v in data_train["Ticket"]])
data_train["SOTON"] = gl.SArray(["SOTON" in v for v in data_train["Ticket"]])
data_train["number"] = gl.SArray([toNumber(v) for v in data_train["Ticket"]])
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Name	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	male	int
	female	int
	no_age	int
	embarked_s	int
	embarked_c	int
	embarked_q	int
	embarked_none	int
	1_class	int
	2_class	int
	3_class	int
	PC	int
	CA	int
	C.A.	int
	W./C.	int
	SOTON	int
	number	int

Rows: 3

Data:
+-------------+----------+-------------------------------+------+-------+-------+
| PassengerId | Survived |              Name             | Age  | SibSp | Parch |
+-------------+----------+-------------------------------+------+-------+-------+
|      1      |    0     |    Braund, Mr. Owen Harris    | 22.0 |   1   |   0   |
|      2      |    1     | Cumings, Mrs. John Bradley... | 38.0 |   1   |   0   |
|      3      |    1     |     Heikkinen, Miss. Laina    | 26.0 |   0   |   0   |
+-------------+----------+-------------------------------+------+-------+-------+
+------------------+---------+-------+------+--------+--------+------------+------------+
|      Ticket      |   Fare  | Cabin | male | female | no_age | embarked_s | embarked_c |
+------------------+---------+-------+------+--------+--------+------------+------------+
|    A/5 21171     |   7.25  |       |  1   |   0    |   0    |     1      |     0      |
|     PC 17599     | 71.2833 |  C85  |  0   |   1    |   0    |     0      |     1      |
| STON/O2. 3101282 |  7.925  |       |  0   |   1    |   0    |     1      |     0      |
+------------------+---------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
|     0      |       0       |    1    |    0    |    0    | 1  | ... |
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
+------------+---------------+---------+---------+---------+----+-----+
[3 rows x 25 columns]

In [10]:
import re

civilite_pattern = re.compile(r" ([A-Za-z])+.")

def get_civilite(name):
    try:
        return civilite_pattern.search(name).group(0)
    except: 
        return ""

civilites_lst = set([get_civilite(v) if get_civilite(v)[-1] == '.' else '' for v in data_train["Name"]])
civilites_lst.remove('')
print civilites_lst

for c in civilites_lst:
    data_train[c] = gl.SArray([get_civilite(v) == c for v in data_train["Name"]])

data_train.head(1)

set([' Miss.', ' Rev.', ' Capt.', ' Mlle.', ' Mrs.', ' Master.', ' Col.', ' Jonkheer.', ' Mr.', ' Ms.', ' Mme.', ' Major.', ' Dr.', ' Don.'])


Columns:
	PassengerId	int
	Survived	int
	Name	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	male	int
	female	int
	no_age	int
	embarked_s	int
	embarked_c	int
	embarked_q	int
	embarked_none	int
	1_class	int
	2_class	int
	3_class	int
	PC	int
	CA	int
	C.A.	int
	W./C.	int
	SOTON	int
	number	int
	 Miss.	int
	 Rev.	int
	 Capt.	int
	 Mlle.	int
	 Mrs.	int
	 Master.	int
	 Col.	int
	 Jonkheer.	int
	 Mr.	int
	 Ms.	int
	 Mme.	int
	 Major.	int
	 Dr.	int
	 Don.	int

Rows: 1

Data:
+-------------+----------+-------------------------+------+-------+-------+
| PassengerId | Survived |           Name          | Age  | SibSp | Parch |
+-------------+----------+-------------------------+------+-------+-------+
|      1      |    0     | Braund, Mr. Owen Harris | 22.0 |   1   |   0   |
+-------------+----------+-------------------------+------+-------+-------+
+-----------+------+-------+------+--------+--------+------------+------------+
|   Ticket  | Fare | Cabin | male | female | no_age | embarked_s | embarked_c |
+-----------+------+-------+------+--------+--------+------------+------------+
| A/5 21171 | 7.25 |       |  1   |   0    |   0    |     1      |     0      |
+-----------+------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
+------------+---------------+---------+---------+---------+----+-----+
[1 rows x 39 columns]

In [11]:
def cabin_letter(cabin):
    try:
        return cabin[0]
    except:
        return ""

cabin_letters = set([cabin_letter(v) for v in data_train["Cabin"]])
cabin_letters.remove('')
print cabin_letters

for c in cabin_letters:
    data_train[c] = gl.SArray([cabin_letter(v) == c for v in data_train["Cabin"]])

data_train.column_names()

set(['A', 'C', 'B', 'E', 'D', 'G', 'F', 'T'])


['PassengerId',
 'Survived',
 'Name',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'male',
 'female',
 'no_age',
 'embarked_s',
 'embarked_c',
 'embarked_q',
 'embarked_none',
 '1_class',
 '2_class',
 '3_class',
 'PC',
 'CA',
 'C.A.',
 'W./C.',
 'SOTON',
 'number',
 ' Miss.',
 ' Rev.',
 ' Capt.',
 ' Mlle.',
 ' Mrs.',
 ' Master.',
 ' Col.',
 ' Jonkheer.',
 ' Mr.',
 ' Ms.',
 ' Mme.',
 ' Major.',
 ' Dr.',
 ' Don.',
 'A',
 'C',
 'B',
 'E',
 'D',
 'G',
 'F',
 'T']

In [12]:
def cabin_number(cabin):
    return toNumber(cabin)

data_train["cabin_number"] = gl.SArray([cabin_number(v) for v in data_train["Cabin"]])

In [33]:
train_set_1, train_set_2 = data_train.random_split(.8)

In [14]:
print train_set_1.head(1)
features = ["Age", "SibSp", "Parch", "Fare", "male", "female", "no_age", 
            "embarked_s", "embarked_c", "embarked_q", "embarked_none",
            "1_class", "2_class", "3_class", 
            "CA", "C.A.", "W./C.", "SOTON", 
            "cabin_number"] + list(civilites_lst) + list(cabin_letters) #, "number"]

+-------------+----------+-------------------------+------+-------+-------+
| PassengerId | Survived |           Name          | Age  | SibSp | Parch |
+-------------+----------+-------------------------+------+-------+-------+
|      1      |    0     | Braund, Mr. Owen Harris | 22.0 |   1   |   0   |
+-------------+----------+-------------------------+------+-------+-------+
+-----------+------+-------+------+--------+--------+------------+------------+
|   Ticket  | Fare | Cabin | male | female | no_age | embarked_s | embarked_c |
+-----------+------+-------+------+--------+--------+------------+------------+
| A/5 21171 | 7.25 |       |  1   |   0    |   0    |     1      |     0      |
+-----------+------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|   

##Create logistic model

In [15]:
#help(gl.classifier.logistic_classifier.create)

In [34]:
simple_logistic_classifier = gl.classifier.logistic_classifier.create(train_set_1, target="Survived", 
                                                                      features=features, validation_set=train_set_2)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 726

Number of classes           : 2

...

| 4         | 5        | 0.027513     | 0.841598          | 0.812121            |

| 5         | 6        | 0.033767     | 0.841598          | 0.812121            |

| 6         | 7        | 0.038965     | 0.841598          | 0.812121            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create SVM model

In [35]:
simple_svm_classifier = gl.classifier.svm_classifier.create(train_set_1, target="Survived", 
                                                            features=features, validation_set=train_set_2, 
                                                            max_iterations=1000)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

SVM:

--------------------------------------------------------

Number of examples          : 726

Number of classes           : 2

...

| 11        | 16       | 1.000000  | 0.050618     | 0.851240          | 0.824242            |

| 51        | 66       | 0.500000  | 0.212055     | 0.844353          | 0.824242            |

| 101       | 149      | 0.250000  | 0.430480     | 0.844353          | 0.824242            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create a decision tree model

In [36]:
decision_tree_model = gl.decision_tree_classifier.create(train_set_1, validation_set=train_set_2,
                                                               target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 726

Number of classes           : 2

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.006789     | 0.888430          | 0.543812          | 0.824242            | 0.562930            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Boosted Tree model

In [37]:
boosted_tree_model = gl.classifier.boosted_trees_classifier.create(train_set_1, validation_set=train_set_2,
                                                                   target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 726

Number of classes           : 2

...

| 3         | 0.020145     | 0.891185          | 0.403053          | 0.836364            | 0.452951            |

| 4         | 0.025106     | 0.900826          | 0.363194          | 0.836364            | 0.434464            |

| 5         | 0.030710     | 0.909091          | 0.327662          | 0.836364            | 0.416233            |

| 6         | 0.036800     | 0.920110          | 0.304839          | 0.836364            | 0.413776            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Random Forest model

In [38]:
random_forest_model = gl.classifier.random_forest_classifier.create(train_set_1, validation_set=train_set_2,
                                                                    target="Survived", features=features, num_trees=100)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Random forest classifier:

--------------------------------------------------------

Number of examples          : 726

Number of classes           : 2

...

| 5         | 0.026616     | 0.884297          | 0.665970          | 0.842424            | 0.668126            |

| 6         | 0.031530     | 0.891185          | 0.660692          | 0.830303            | 0.663256            |

| 11        | 0.053991     | 0.893939          | 0.635446          | 0.818182            | 0.641393            |

| 51        | 0.226360     | 0.898072          | 0.473822          | 0.830303            | 0.503634            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Cleanning testing data
DO NOT FORGOT ANY CLEANING OPERATION MADE IN THE TRAINING DATA AND USED BY CLASSIFIER !!!

In [21]:
data_test["male"] = data_test["Sex"] == "male"
data_test["female"] = data_test["Sex"] == "female"
data_test = data_test.remove_column("Sex")

In [22]:
data_test["no_age"] = data_test["Age"] == None
data_test["Age"] = gl.SArray([0 if v == None else v for v in data_test["Age"]])

In [23]:
data_test["embarked_s"] = data_test["Embarked"] == "S"
data_test["embarked_c"] = data_test["Embarked"] == "C"
data_test["embarked_q"] = data_test["Embarked"] == "Q"
data_test["embarked_none"] = data_test["Embarked"] == None
data_test = data_test.remove_column("Embarked")

In [24]:
data_test["1_class"] = data_test["Pclass"] == 1
data_test["2_class"] = data_test["Pclass"] == 2
data_test["3_class"] = data_test["Pclass"] == 3
data_test = data_test.remove_column("Pclass")

In [25]:
data_test["number"] = gl.SArray([toNumber(v) for v in data_test["Ticket"]])
data_test["PC"] = gl.SArray(["PC" in v for v in data_test["Ticket"]])
data_test["CA"] = gl.SArray(["CA" in v for v in data_test["Ticket"]])
data_test["C.A."] = gl.SArray(["C.A." in v for v in data_test["Ticket"]])
data_test["W./C."] = gl.SArray(["W./C." in v for v in data_test["Ticket"]])
data_test["SOTON"] = gl.SArray(["SOTON" in v for v in data_test["Ticket"]])
data_test["number"] = gl.SArray([toNumber(v) for v in data_test["Ticket"]])

In [26]:
for c in civilites_lst:
    data_test[c] = gl.SArray([get_civilite(v) == c for v in data_test["Name"]])

In [27]:
for c in cabin_letters:
    data_test[c] = gl.SArray([cabin_letter(v) == c for v in data_test["Cabin"]])

In [29]:
data_test["cabin_number"] = gl.SArray([cabin_number(v) for v in data_test["Cabin"]])

##Making Predictions

In [39]:
data_test["Survived"] = random_forest_model.predict(data_test)
#random_forest_model.predict(data_test)
#boosted_tree_model.predict(data_test)
#decision_tree_model.predict(data_test)
#simple_svm_classifier.predict(data_test) 
#simple_logistic_classifier.predict(data_test)

In [40]:
submission = gl.SFrame()

In [41]:
submission["PassengerId"] = data_test["PassengerId"]
submission["Survived"] = data_test["Survived"]

In [42]:
submission.save("kaggle.csv", format="csv")

In [85]:
#data_train.show()

In [86]:
#data_train.head(10)